In [1]:
import numpy as np
import pandas as pd

import configparser
import pymongo

In [2]:
config = configparser.ConfigParser()
config.read('../database-dev/auth/config.ini')

MONGO_LOCAL = config.get('mongodb', 'MONGO_LOCAL')
MONGO_DB = config.get('mongodb', 'MONGO_DB')
client = pymongo.MongoClient(MONGO_LOCAL)
db = client.get_database(MONGO_DB)
collection = db['reps']

In [208]:
proj_dict = {
    'first_name': 1,
    'middle_name': 1,
    'last_name': 1,
    'dob': 1,
    'gender': 1,
    'current_party': 1,
    'state': 1,
    'roles': 1,
    'education': 1
}
results = collection.find({'in_office': True}, proj_dict)
df = pd.DataFrame(results)

In [209]:
from datetime import date

In [210]:
df['dob'] = df['dob'].map(lambda x: pd.to_datetime(x).date())

In [211]:
def get_age(dob):
    today = date.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    
    return age

In [212]:
df['age'] = np.vectorize(get_age)(df['dob'].values)

In [213]:
df['terms'] = np.vectorize(lambda x: len(x))(df['roles'])

In [214]:
df['education'] = df['education'].map(lambda x: x == None and [['HS', 'HS']] or x)

In [215]:
df['max_edu'] = df['education'].str[-1]

In [222]:
df['max_degree'] = df['max_edu'].str[0]
df['max_institution'] = df['max_edu'].str[1]

In [232]:
df.loc[df['max_degree'] == 'MS']['gender'].value_counts(normalize=True)

M    0.545455
F    0.454545
Name: gender, dtype: float64

In [233]:
df['gender'].value_counts(normalize=True)

M    0.719178
F    0.280822
Name: gender, dtype: float64